In [2]:
#downsampling (giam do phan giai buc anh) 
import cv2
import os

input_folder = "/home/duongnhan/Chon/PMRF/datasets/ct_images/HR"
output_folder = "/home/duongnhan/Chon/PMRF/datasets/ct_images/LR/"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    img = cv2.imread(os.path.join(input_folder, filename), cv2.IMREAD_GRAYSCALE)
    img_lr = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(os.path.join(output_folder, filename), img_lr)

print("Tạo ảnh LR hoàn tất!")


Tạo ảnh LR hoàn tất!


In [13]:
# them nhieu gausian

import numpy as np

def add_gaussian_noise(image, mean=0, var=10):
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, gaussian)
    return noisy_image

def add_motion_artifact(image, kernel_size=15):
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size - 1)/2), :] = np.ones(kernel_size)
    kernel = kernel / kernel_size
    motion_blur = cv2.filter2D(image, -1, kernel)
    return motion_blur

def add_LR(input_folder, output_folder):
    os.makedirs(f"{output_folder}/gaussian_noise", exist_ok=True)
    os.makedirs(f"{output_folder}/motion", exist_ok=True)
    for filename in os.listdir(input_folder):
        img = cv2.imread(os.path.join(input_folder, filename), cv2.IMREAD_GRAYSCALE)
        img_noisy = add_gaussian_noise(img)
        cv2.imwrite(os.path.join(f"{output_folder}/gaussian_noise", filename), img_noisy)

        img_motion = add_motion_artifact(img)
        cv2.imwrite(os.path.join(f"{output_folder}/motion", filename), img_motion)

        print(f"add noise sucessful image: {filename}" )
    print("Thêm nhiễu Gaussian hoàn tất!")

In [22]:
input_folder = "./datasets/data1/head1/"
output_folder = "./data/LRimg/head1"
add_LR(input_folder, output_folder)

add noise sucessful image: image_s0003_i0055.jpg
add noise sucessful image: image_s0003_i0027.jpg
add noise sucessful image: image_s0003_i0077.jpg
add noise sucessful image: image_s0003_i0041.jpg
add noise sucessful image: image_s0003_i0043.jpg
add noise sucessful image: image_s0003_i0045.jpg
add noise sucessful image: image_s0003_i0069.jpg
add noise sucessful image: image_s0003_i0105.jpg
add noise sucessful image: image_s0003_i0071.jpg
add noise sucessful image: image_s0003_i0024.jpg
add noise sucessful image: image_s0003_i0080.jpg
add noise sucessful image: image_s0003_i0115.jpg
add noise sucessful image: image_s0003_i0037.jpg
add noise sucessful image: image_s0003_i0006.jpg
add noise sucessful image: image_s0003_i0042.jpg
add noise sucessful image: image_s0003_i0022.jpg
add noise sucessful image: image_s0003_i0017.jpg
add noise sucessful image: image_s0003_i0016.jpg
add noise sucessful image: image_s0003_i0079.jpg
add noise sucessful image: image_s0003_i0060.jpg
add noise sucessful 

add noise sucessful image: image_s0003_i0031.jpg
add noise sucessful image: image_s0003_i0004.jpg
add noise sucessful image: image_s0003_i0107.jpg
add noise sucessful image: image_s0003_i0052.jpg
add noise sucessful image: image_s0003_i0025.jpg
add noise sucessful image: image_s0003_i0059.jpg
add noise sucessful image: image_s0003_i0082.jpg
add noise sucessful image: image_s0003_i0083.jpg
add noise sucessful image: image_s0003_i0087.jpg
add noise sucessful image: image_s0003_i0098.jpg
add noise sucessful image: image_s0003_i0035.jpg
add noise sucessful image: image_s0003_i0056.jpg
add noise sucessful image: image_s0003_i0092.jpg
add noise sucessful image: image_s0003_i0086.jpg
add noise sucessful image: image_s0003_i0106.jpg
add noise sucessful image: image_s0003_i0019.jpg
add noise sucessful image: image_s0003_i0013.jpg
add noise sucessful image: image_s0003_i0057.jpg
add noise sucessful image: image_s0003_i0046.jpg
add noise sucessful image: image_s0003_i0049.jpg
add noise sucessful 

In [23]:
import os
import shutil
import random
#778 anh

def split_dataset(input_GT, input_LQ, output_folder, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    assert train_ratio + val_ratio + test_ratio == 1.0, "Tổng tỷ lệ phải bằng 1"

    # Tạo thư mục đầu ra
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(output_folder, split, "GT"), exist_ok=True)
        os.makedirs(os.path.join(output_folder, split, "LQ"), exist_ok=True)

    # Lấy danh sách ảnh và shuffle để đảm bảo dữ liệu phân bố ngẫu nhiên
    gt_images = sorted(os.listdir(input_GT))
    lq_images = sorted(os.listdir(input_LQ))  # Giữ thứ tự giống GT
    assert len(gt_images) == len(lq_images), "Số lượng ảnh GT và LQ không khớp!"

    random.shuffle(gt_images)  # Xáo trộn ảnh để chia ngẫu nhiên

    # Tính số lượng ảnh cho mỗi tập
    total_images = len(gt_images)
    train_count = int(total_images * train_ratio)
    val_count = int(total_images * val_ratio)
    test_count = total_images - train_count - val_count  # Phần còn lại cho test

    # Chia ảnh thành 3 phần
    train_files = gt_images[:train_count]
    val_files = gt_images[train_count:train_count + val_count]
    test_files = gt_images[train_count + val_count:]

    # Hàm sao chép ảnh vào thư mục tương ứng
    def copy_files(file_list, split_name):
        for file in file_list:
            shutil.copy(os.path.join(input_GT, file), os.path.join(output_folder, split_name, "GT", file))
            shutil.copy(os.path.join(input_LQ, file), os.path.join(output_folder, split_name, "LQ", file))

    # Copy ảnh vào thư mục train, val, test
    copy_files(train_files, "train")
    copy_files(val_files, "val")
    copy_files(test_files, "test")

    print(f"✅ Đã chia tập dữ liệu: Train ({train_count}), Val ({val_count}), Test ({test_count})")

# Gọi hàm với đường dẫn thư mục GT, LQ và thư mục đầu ra
split_dataset("./datasets/GT", "./datasets/LQ", "./datasets/")


✅ Đã chia tập dữ liệu: Train (548), Val (68), Test (69)


In [20]:
import os
import re

def rename_images(folder_path, new_s_value="s0004"):
    # Lặp qua tất cả các tệp trong thư mục
    for filename in os.listdir(folder_path):
        match = re.match(r"(image_s\d{4}_i\d{4}\.jpg)", filename)
        if match:
            # Tách phần "sXXXX" và "iYYYY"
            parts = filename.split("_")
            new_filename = f"{parts[0]}_{new_s_value}_{parts[2]}"
            
            # Đường dẫn đầy đủ
            old_path = os.path.join(folder_path, filename)
            new_path = os.path.join(folder_path, new_filename)

            # Đổi tên file
            os.rename(old_path, new_path)
            print(f"Đổi: {filename} ➝ {new_filename}")

# Thư mục chứa ảnh cần đổi tên
folder_path = "./datasets/data1/head2/"
rename_images(folder_path)


Đổi: image_s0001_i0114.jpg ➝ image_s0004_i0114.jpg
Đổi: image_s0001_i0099.jpg ➝ image_s0004_i0099.jpg
Đổi: image_s0001_i0101.jpg ➝ image_s0004_i0101.jpg
Đổi: image_s0001_i0048.jpg ➝ image_s0004_i0048.jpg
Đổi: image_s0001_i0028.jpg ➝ image_s0004_i0028.jpg
Đổi: image_s0001_i0116.jpg ➝ image_s0004_i0116.jpg
Đổi: image_s0001_i0105.jpg ➝ image_s0004_i0105.jpg
Đổi: image_s0001_i0031.jpg ➝ image_s0004_i0031.jpg
Đổi: image_s0001_i0037.jpg ➝ image_s0004_i0037.jpg
Đổi: image_s0001_i0057.jpg ➝ image_s0004_i0057.jpg
Đổi: image_s0001_i0124.jpg ➝ image_s0004_i0124.jpg
Đổi: image_s0001_i0113.jpg ➝ image_s0004_i0113.jpg
Đổi: image_s0001_i0004.jpg ➝ image_s0004_i0004.jpg
Đổi: image_s0001_i0078.jpg ➝ image_s0004_i0078.jpg
Đổi: image_s0001_i0012.jpg ➝ image_s0004_i0012.jpg
Đổi: image_s0001_i0065.jpg ➝ image_s0004_i0065.jpg
Đổi: image_s0001_i0064.jpg ➝ image_s0004_i0064.jpg
Đổi: image_s0001_i0005.jpg ➝ image_s0004_i0005.jpg
Đổi: image_s0001_i0090.jpg ➝ image_s0004_i0090.jpg
Đổi: image_s0001_i0010.jpg ➝ im

In [1]:
import torch
torch.cuda.empty_cache()
